# **Database cleanup steps** 

Any queries or shaping of the dataset is done to render the data, making it more similar to other datasets used in this project.

**1.**

importing dependencies is the crucial first step in data manipulation using pandas.


In [1]:
import pandas as pd
from config import path

**2.**

Then the original csv must be located.
File is located in zip on github

In [2]:
terrorism_csv = "globalterrorismdb_0718dist.csv"

**3.**

Following that, the data is read and renamed.

In [3]:
terrorism_data = pd.read_csv(terrorism_csv, encoding="ISO-8859-1", low_memory=False)

**4.**

A dataframe is then created using the particular columns, which will be beneficial when combining with other datasets.

In [4]:
terrorism_df = terrorism_data[['iyear', 'imonth', 'country_txt', 'region_txt', 'gname', 'attacktype1_txt', 'weaptype1_txt']]

**5.**

The first 5 rows of the new terrorism dataframe is then outputted for visual inspection.

In [5]:
terrorism_df.head(5)

,iyear,imonth,country_txt,region_txt,gname,attacktype1_txt,weaptype1_txt
0,1970,7,Dominican Republic,Central America & Caribbean,MANO-D,Assassination,Unknown
1,1970,0,Mexico,North America,23rd of September Communist League,Hostage Taking (Kidnapping),Unknown
2,1970,1,Philippines,Southeast Asia,Unknown,Assassination,Unknown
3,1970,1,Greece,Western Europe,Unknown,Bombing/Explosion,Explosives
4,1970,1,Japan,East Asia,Unknown,Facility/Infrastructure Attack,Incendiary


**6.**

The types of the values are then checked to ensure they are compatiable with the other data going into the rational database.

In [6]:
terrorism_df.dtypes

iyear               int64
imonth              int64
country_txt        object
region_txt         object
gname              object
attacktype1_txt    object
weaptype1_txt      object
dtype: object

**7.**

The shape of the dataframe is then checked.

In [7]:
terrorism_df.shape

(181691, 7)

**8.**

Any NULL values are checked.

In [8]:
terrorism_df.isnull().any()

iyear              False
imonth             False
country_txt        False
region_txt         False
gname              False
attacktype1_txt    False
weaptype1_txt      False
dtype: bool

**9.**

A query is then done to isolate the years between 1983 and 2013. Data associated with the year 1983 is only taken from the month of May on.


In [9]:
terrorism_df= terrorism_df.query('(iyear > 1983 & iyear < 2013) or (iyear == 1983 & imonth >= 5) or (iyear == 2013 & imonth < 5)') 

**10.**

The shape of the dataframe is then checked once again.

In [10]:
terrorism_df.shape

(97973, 7)

**11.**

The dataframe is then printed to a csv.

In [11]:
terrorism_df.to_csv('terrorismCleanData.csv', index=False)

# USD cleanup

## Chart retrieved from
https://www.macrotrends.net/1329/us-dollar-index-historical-chart 
Had to ignore a large header before the data

In [12]:
data = pd.read_csv('us-dollar-index-historical-chart.csv', skiprows = 15)

dfUSD = data.copy()
dfUSD.head()

,date,value
0,1973-01-01,107.616
1,1973-02-01,103.046
2,1973-03-01,100.000
3,1973-04-01,100.376
4,1973-05-01,99.263


## There were empty rows so we got rid of those

In [13]:
dfUSD = dfUSD.dropna()

In [14]:
dfUSD.shape

(564, 2)

## Reformatting the date to better match the other data

In [15]:
for index, row in dfUSD.iterrows():
    strDate = row['date']
    split = strDate.split('-')
    dfUSD.at[index, 'year'] = int(split[0])
    dfUSD.at[index, 'month'] = int(split[1])

In [16]:
dfUSD.head()

,date,value,year,month
0,1973-01-01,107.616,1973.0,1.0
1,1973-02-01,103.046,1973.0,2.0
2,1973-03-01,100.000,1973.0,3.0
3,1973-04-01,100.376,1973.0,4.0
4,1973-05-01,99.263,1973.0,5.0


## We only wanted the 30 year span
so we trim the dataframe

In [17]:
dfUSDTrimmed = dfUSD.query('(year > 1983 & year < 2013) or (year == 1983 & month >= 5) or (year == 2013 & month < 5)')
dfUSDTrimmed.shape

(360, 4)

## The value column had an extra space. 
Not really required but makes it nicer

In [18]:
dfUSDTrimmed.rename(columns = {' value': 'value'}, inplace = True)

C:\Users\caroj\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


## Changing our year and month to be ints

In [19]:
dfUSDTrimmed['year'] = dfUSDTrimmed['year'].astype('int64')
dfUSDTrimmed['month'] = dfUSDTrimmed['month'].astype('int64')

dfUSDTrimmed.head()

C:\Users\caroj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\caroj\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date,value,year,month
124,1983-05-01,109.292,1983,5
125,1983-06-01,110.775,1983,6
126,1983-07-01,111.495,1983,7
127,1983-08-01,113.021,1983,8
128,1983-09-01,113.237,1983,9


# No longer care about the date column and want to rearrange the columns

In [20]:
dfUSDTrimmed = dfUSDTrimmed[['year', 'month', 'value']]
dfUSDTrimmed.head()

,year,month,value
124,1983,5,109.292
125,1983,6,110.775
126,1983,7,111.495
127,1983,8,113.021
128,1983,9,113.237


In [21]:
dfUSDTrimmed.shape

(360, 3)

# Export this guy to be imported into sql

In [22]:
dfUSDTrimmed.to_csv('USDClean.csv', index = False)

# Oil cleanup

Oil Price Data Cleanup from May 1983 to May 2013 by Mark McPherson
Original CSV file from www.investing.com/commodities/crude-oil-historical-data

In [23]:
#import original CSV
import pandas as pd
import numpy as np

data = pd.read_csv('oil_data_83_13_orig.csv')

In [24]:
oil_price_df = data.copy()
oil_price_df.head(5)

,Date,Price,Open,High,Low,Vol.,Change %
0,Apr 13,93.46,97.36,97.80,85.61,5.43M,-3.88%
1,Mar 13,97.23,91.76,97.35,89.33,4.03M,5.63%
2,Feb 13,92.05,97.42,98.15,91.57,4.12M,-5.58%
3,Jan 13,97.49,91.78,98.24,91.52,4.38M,6.18%
4,Dec 12,91.82,88.85,91.99,85.21,3.59M,3.27%


In [25]:
#Pare dataframe down to just Date and Price
oil_price_df = oil_price_df[['Date', 'Price']]

In [26]:
#Split Date into 3 char string for Month and 2 for Year in sep. columns
# and drop former Date
new = oil_price_df["Date"].str.split(" ", n = 1, expand = True) 

oil_price_df["Month"]= new[0] 
  
oil_price_df["Year"]= new[1]

oil_price_df.drop(columns =["Date"], inplace = True)

In [27]:
oil_price_df = oil_price_df[['Month', 'Year', 'Price']]

In [28]:
oil_price_rev_df = oil_price_df.iloc[::-1]
oil_price_rev_df = oil_price_rev_df.reset_index(drop=True)
oil_price_rev_df

,Month,Year,Price
0,May,83,30.25
1,Jun,83,31.38
2,Jul,83,32.00
3,Aug,83,31.59
4,Sep,83,30.36
...,...,...,...
355,Dec,12,91.82
356,Jan,13,97.49
357,Feb,13,92.05
358,Mar,13,97.23


In [29]:
#Convert Year to an integer
oil_price_rev_df['Year'] = oil_price_rev_df['Year'].astype(int)

In [30]:
#Conversion to 4 digit year - add 1900 if > 50, 2000 if not
for index, row in oil_price_rev_df.iterrows():
    year = row.loc['Year']
    if year > 50:
        oil_price_rev_df.at[index, 'Year'] = year + 1900
    else:
        oil_price_rev_df.at[index, 'Year'] = year + 2000

In [31]:
oil_price_rev_df

,Month,Year,Price
0,May,1983,30.25
1,Jun,1983,31.38
2,Jul,1983,32.00
3,Aug,1983,31.59
4,Sep,1983,30.36
...,...,...,...
355,Dec,2012,91.82
356,Jan,2013,97.49
357,Feb,2013,92.05
358,Mar,2013,97.23


In [32]:
# Map the 3 char Month to an integer
months = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
oil_price_rev_df.Month = oil_price_rev_df.Month.map(months)

In [33]:
oil_price_rev_df

,Month,Year,Price
0,5,1983,30.25
1,6,1983,31.38
2,7,1983,32.00
3,8,1983,31.59
4,9,1983,30.36
...,...,...,...
355,12,2012,91.82
356,1,2013,97.49
357,2,2013,92.05
358,3,2013,97.23


In [34]:
oil_price_rev_df.to_csv('oil_data_clean.csv', index=False)

# SQL queries
## import ETL.tar into an sql server named ETL

In [35]:
from sqlalchemy import create_engine

In [36]:
databaseName = "ETL"
engine = create_engine(path + databaseName)

In [37]:
connection = engine.connect()

In [38]:
dfUSDSQL = pd.read_sql('select * from USD', connection)
dfUSDSQL.head()

,year,month,dollarindex
0,1983,5,109.292
1,1983,6,110.775
2,1983,7,111.495
3,1983,8,113.021
4,1983,9,113.237


In [39]:
dfTerrorism = pd.read_sql('select * from terrorism', connection)

dfTerrorism.head()

,year,month,country,region,groupname,attacktype,weaptype
0,1986,2,Sweden,Western Europe,Unknown,Armed Assault,Firearms
1,1983,5,Lebanon,Middle East & North Africa,Unknown,Assassination,Explosives
2,1983,5,El Salvador,Central America & Caribbean,Farabundo Marti National Liberation Front (FMLN),Armed Assault,Firearms
3,1983,5,Lebanon,Middle East & North Africa,Unknown,Bombing/Explosion,Explosives
4,1983,5,Lebanon,Middle East & North Africa,Unknown,Bombing/Explosion,Unknown


In [40]:
dfOil = pd.read_sql('select * from oil', connection)
dfOil.head()

,month,year,price
0,5,1983,30.25
1,6,1983,31.38
2,7,1983,32.00
3,8,1983,31.59
4,9,1983,30.36


In [41]:
dfMerged = pd.merge(dfTerrorism, dfUSDSQL, how = 'left', left_on = ['year', 'month'], right_on = ['year', 'month'])


In [42]:
dfMerged = pd.merge(dfMerged, dfOil, how = 'left', left_on = ['year', 'month'], right_on = ['year', 'month'])


In [43]:
dfMerged.head()

,year,month,country,region,groupname,attacktype,weaptype,dollarindex,price
0,1986,2,Sweden,Western Europe,Unknown,Armed Assault,Firearms,111.414,13.26
1,1983,5,Lebanon,Middle East & North Africa,Unknown,Assassination,Explosives,109.292,30.25
2,1983,5,El Salvador,Central America & Caribbean,Farabundo Marti National Liberation Front (FMLN),Armed Assault,Firearms,109.292,30.25
3,1983,5,Lebanon,Middle East & North Africa,Unknown,Bombing/Explosion,Explosives,109.292,30.25
4,1983,5,Lebanon,Middle East & North Africa,Unknown,Bombing/Explosion,Unknown,109.292,30.25


## seeing the data for the year before and after 9/11

In [44]:
dfQuery = dfMerged.query('(year == 2001) or (year == 2002 & month < 9) or (year == 2000 & month >= 9)')

dfQuery.head()

,year,month,country,region,groupname,attacktype,weaptype,dollarindex,price
51369,2000,12,Nepal,South Asia,Unknown,Bombing/Explosion,Explosives,107.105,26.80
52223,2000,9,Kenya,Sub-Saharan Africa,Bandits,Armed Assault,Firearms,105.843,30.84
52254,2000,9,Uganda,Sub-Saharan Africa,Lord's Resistance Army (LRA),Unknown,Unknown,105.843,30.84
52255,2000,9,Sri Lanka,South Asia,Liberation Tigers of Tamil Eelam (LTTE),Bombing/Explosion,Explosives,105.843,30.84
52256,2000,9,Guinea,Sub-Saharan Africa,Unknown,Armed Assault,Firearms,105.843,30.84


In [45]:
gPrices = dfQuery.groupby(['year', 'month'])

gPrices.mean()

dollarindex  price
year month                    
2000 9          105.843  30.84
     10         107.344  32.70
     11         107.956  33.82
     12         107.105  26.80
2001 1          107.458  28.66
     2          108.267  27.39
     3          109.861  26.29
     4          110.499  28.46
     5          110.543  28.37
     6          111.364  26.25
     7          111.493  26.35
     8          109.517  27.20
     9          110.129  23.43
     10         110.519  21.18
     11         110.999  19.44
     12         110.944  19.84
2002 1          112.174  19.48
     2          112.767  21.74
     3          112.155  26.31
     4          111.927  27.29
     5          110.419  25.31
     6          108.955  26.86
     7          107.364  27.02
     8          108.653  28.98

## the following queries can be graphed with oil/usd to see see correlation

### amount of attacks in the US per year

In [46]:
dfQuery2 = dfMerged.query('country == "United States"')

gUS = dfQuery2.groupby('year')

gUS.count()

,month,country,region,groupname,attacktype,weaptype,dollarindex,price
year,,,,,,,,
1983,29,29,29,29,29,29,29,29
1984,63,63,63,63,63,63,63,63
1985,40,40,40,40,40,40,40,40
1986,49,49,49,49,49,49,49,49
1987,34,34,34,34,34,34,34,34
1988,27,27,27,27,27,27,27,27
1989,42,42,42,42,42,42,42,42
1990,32,32,32,32,32,32,32,32
1991,30,30,30,30,30,30,30,30


### seeing the attacks in the middle east

In [47]:
dfQuery3 = dfMerged.query('region == "Middle East & North Africa"')

gCountry = dfQuery3.groupby(['year', 'month', 'country'])
gCountry.count()

region  groupname  attacktype  weaptype  \
year month country                                                             
1983 5     Israel                         1          1           1         1   
           Jordan                         2          2           2         2   
           Lebanon                       28         28          28        28   
           West Bank and Gaza Strip       1          1           1         1   
     6     Israel                         7          7           7         7   
...                                     ...        ...         ...       ...   
2013 4     Libya                         19         19          19        19   
           Syria                         22         22          22        22   
           Tunisia                        3          3           3         3   
           West Bank and Gaza Strip       6          6           6         6   
           Yemen                         28         28          28        28   

                                     dollarindex  price  
year month country                                       
1983 5     Israel                              1      1  
           Jordan                              2      2  
           Lebanon                            28     28  
           West Bank and Gaza Strip            1      1  
     6     Israel                              7      7  
...                                          ...    ...  
2013 4     Libya                              19     19  
           Syria                              22     22  
           Tunisia                             3      3  
           West Bank and Gaza Strip            6      6  
           Yemen                              28     28  

[2191 rows x 6 columns]

## Attacks using explosives in the US

In [48]:
dfQuery4 = dfMerged.query('country == "United States" & weaptype == "Explosives"')

gExp = dfQuery4.groupby(['year', 'month'])

gExp.count()

country  region  groupname  attacktype  weaptype  dollarindex  \
year month                                                                  
1983 5            3       3          3           3         3            3   
     7            1       1          1           1         1            1   
     8            2       2          2           2         2            2   
     10           2       2          2           2         2            2   
     11           2       2          2           2         2            2   
...             ...     ...        ...         ...       ...          ...   
2011 1            2       2          2           2         2            2   
     5            1       1          1           1         1            1   
2012 6            1       1          1           1         1            1   
2013 1            1       1          1           1         1            1   
     4            4       4          4           4         4            4   

            price  
year month         
1983 5          3  
     7          1  
     8          2  
     10         2  
     11         2  
...           ...  
2011 1          2  
     5          1  
2012 6          1  
2013 1          1  
     4          4  

[140 rows x 7 columns]